In [1]:
import numpy as np
import math
import keras
import tensorflow as tf
from cleverhans.utils_mnist import data_mnist
from cleverhans.utils import batch_indices
from cleverhans.attacks import DeepFool
from cleverhans.utils_keras import KerasModelWrapper

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten

/Users/kojin/anaconda/envs/ml/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/kojin/anaconda/envs/ml/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [73]:
lr = 0.001
epochs = 10
batch_size = 128 
drop_ratio = 0.10
num_inputs = int(28 * 28 * (1-drop_ratio))
noise_eps = 0.2

In [3]:
def logistic_regression_model(input_ph,num_inputs, nb_classes=10):
    model = Sequential()
    model.add(Dense(nb_classes,input_shape=(num_inputs,)))
    model.add(Activation('softmax'))
    return model

In [65]:
# Create TF session and set as Keras backend session
sess = tf.Session()
# Get MNIST test data
X_train, Y_train, X_test, Y_test = data_mnist()
# Flatten image matrices
num_train, num_rows, num_cols, num_channels = X_train.shape
num_test, _, _, _ = X_test.shape
X_train = X_train.reshape((num_train, num_rows * num_cols * num_channels))
X_test = X_test.reshape((num_test, num_rows * num_cols * num_channels))
# Randomly drop some ratio of pixels
input_indices = np.random.choice(num_rows * num_cols, num_inputs, replace = False)
X_train = X_train[:,input_indices]
X_test = X_test[:,input_indices]

In [77]:
# Define placeholders
x = tf.placeholder(tf.float32, shape=(None, num_inputs))
y = tf.placeholder(tf.float32, shape=(None, 10))

# Wrap log reg model for applying adversarial examples
model = logistic_regression_model(x,num_inputs)
wrap = KerasModelWrapper(model, 10) 

# Define the objective
logits = wrap.get_logits(x)
loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=logits)

# Define the update func
optimizer = tf.train.AdamOptimizer(learning_rate=lr)
train_step = optimizer.minimize(loss)
acc, acc_op = tf.metrics.accuracy(labels=tf.argmax(y, 1), 
                                  predictions=tf.argmax(logits,1))

# Define adv attack
deepfool = DeepFool(wrap, sess=sess)
deepfool_params = {'eps': noise_eps,
               'clip_min': 0.,
               'clip_max': 1.}

# Attack images
adv_x = deepfool.generate(x, **deepfool_params)
# Consider the attack to be constant
adv_x = tf.stop_gradient(adv_x)

# Evaluate predictions on adv attacks
preds = model(adv_x)
acc_adv, acc_op_adv = tf.metrics.accuracy(labels=tf.argmax(y, 1), 
                                  predictions=tf.argmax(preds,1))

In [78]:
# Initialize variables
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())
rng = np.random.RandomState() # for batch sampling

for epoch in range(epochs):
    # Compute number of batches
    num_batches = int(math.ceil(float(len(X_train)) / batch_size))
    assert num_batches * batch_size >= len(X_train)

    # Indices to shuffle training set
    index_shuf = list(range(len(X_train)))
    rng.shuffle(index_shuf)
    for batch in range(num_batches):

        # Compute batch start and end indices
        start, end = batch_indices(
            batch, len(X_train), batch_size)

        # Perform one training step
        feed_dict = {x: X_train[index_shuf[start:end]],
                     y: Y_train[index_shuf[start:end]]}
        sess.run(train_step,feed_dict=feed_dict)
    
    feed_dict = {x: X_train,
                 y: Y_train}
    acc_val = sess.run(acc_op,feed_dict=feed_dict)
    print(f"Epoch: {epoch}, Train Acc: {acc_val:.5f}")

Epoch: 0, Train Acc: 0.88877
Epoch: 1, Train Acc: 0.89706
Epoch: 2, Train Acc: 0.90278
Epoch: 3, Train Acc: 0.90650
Epoch: 4, Train Acc: 0.90930
Epoch: 5, Train Acc: 0.91142
Epoch: 6, Train Acc: 0.91310
Epoch: 7, Train Acc: 0.91459
Epoch: 8, Train Acc: 0.91579
Epoch: 9, Train Acc: 0.91685


In [80]:
# Evaluate the model on the test set
feed_dict = {x: X_test,
             y: Y_test}
sess.run(acc_op,feed_dict=feed_dict)

0.9169869

In [79]:
feed_dict = {x: X_test,
             y: Y_test}
sess.run(acc_op_adv,feed_dict=feed_dict)

0.046